In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sparrow as sp

In [ ]:
import os

from spatialdata import read_zarr

from sparrow.table.cell_clustering._utils import _export_to_ark_format as _export_to_ark_format_cells
from sparrow.table.pixel_clustering._cluster_intensity import _export_to_ark_format as _export_to_ark_format_pixels
from sparrow.utils._keys import ClusteringKey

# .zarr available at:
#https://drive.google.com/file/d/1Wn8Th1TjqcWCtQF5gA23s2MXjqoryHJR/view?usp=drive_link
# TODO do it via https://github.com/fatiando/pooch

path = "/Users/arnedf/VIB/DATA/ark_analysis"

sdata_ark_analysis = read_zarr( os.path.join( path, "sdata_ark.zarr" ) )

channels=["CD3", "CD4", "CD8", "CD14", "CD20", "CD31", "CD45", "CD68",
            "CD163", "CK17", "Collagen1", "Fibronectin",
            "ECAD", "HLADR", "SMA", "Vim"]

sdata_ark_analysis=sp.im.pixel_clustering_preprocess(
    sdata_ark_analysis,
    img_layer=["raw_image_fov0", "raw_image_fov1"  ],
    output_layer=[ "raw_image_fov0_processed", "raw_image_fov1_processed" ],
    channels=channels,
    chunks=2048,
    overwrite=True,
    sigma=2.0,
)


sdata_ark_analysis, fsom, mapping=sp.im.flowsom(
    sdata_ark_analysis,
    img_layer=[ "raw_image_fov0_processed", "raw_image_fov1_processed" ],
    output_layer_clusters=[ "raw_image_fov0_flowsom_clusters", "raw_image_fov1_flowsom_clusters" ], # we need output_cluster_layer and output_meta_cluster_layer --> these will both be labels layers
    output_layer_metaclusters=[ "raw_image_fov0_flowsom_metaclusters",  "raw_image_fov1_flowsom_metaclusters" ],
    n_clusters=20,
    random_state=111,
    chunks=512,
    overwrite=True,
 )

sdata_ark_analysis=sp.tb.cluster_intensity(
    sdata_ark_analysis,
    mapping=mapping,
    img_layer=["raw_image_fov0_processed", "raw_image_fov1_processed" ],
    labels_layer=[ "raw_image_fov0_flowsom_clusters", "raw_image_fov1_flowsom_clusters" ],
    #labels_layer="raw_image_fov0_flowsom_clusters",
    output_layer="counts_clusters",
    overwrite=True,
        )
# TODO: to visualize in napari spatialdata, this counts_clusters table layer needs to be removed from the sdata, because napari-spatialdata does not support viewing sdata that contain tables not linked to element.

sdata_ark_analysis, fsom = sp.tb.flowsom(
    sdata_ark_analysis,
    labels_layer_cells=["label_whole_fov0", "label_whole_fov1" ],
    labels_layer_clusters=["raw_image_fov0_flowsom_metaclusters", "raw_image_fov1_flowsom_metaclusters"],  # here you could also choose "ark_pixel_som_cluster"
    output_layer="table_cell_clustering_flowsom",
    chunks=512,
    overwrite=True,
    random_state=100,
)


# weighted channel average for visualization -> calculate this on the flowsom clustered matrix
sdata_ark_analysis=sp.tb.weighted_channel_expression(
    sdata_ark_analysis,
    table_layer_cell_clustering="table_cell_clustering_flowsom",
    table_layer_pixel_cluster_intensity="counts_clusters",
    output_layer="table_cell_clustering_flowsom",
    clustering_key=ClusteringKey._METACLUSTERING_KEY,
    overwrite=True,
                      )


df=_export_to_ark_format_pixels( adata=sdata_ark_analysis[ "counts_clusters" ], output=path )

df_cell_som_cluster_count_avg, df_cell_som_cluster_channel_avg, df_cell_meta_cluster_channel_avg=_export_to_ark_format_cells( sdata_ark_analysis, table_layer="table_cell_clustering_flowsom" , output=path )